# Titanic

## Librerie

In [214]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sl

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

print(tf.__version__)
print(pd.__version__)
print(np.__version__)
print(sl.__version__)

2.2.0
1.0.4
1.18.4
0.22.2.post1


## Download dei Dataset

Importiamo i dataset necessari per l'addestramento e per il test della rete neurale:

In [0]:
url_train = 'https://raw.githubusercontent.com/rirolli/Titanic/master/train.csv'
url_test = 'https://raw.githubusercontent.com/rirolli/Titanic/master/test.csv'
url_test_y = 'https://raw.githubusercontent.com/rirolli/Titanic/master/gender_submission.csv'

titanic_load_train = pd.read_csv(url_train)
titanic_load_test = pd.read_csv(url_test)
titanic_load_test_label = pd.read_csv(url_test_y)

In [216]:
# Stampa delle prima 5 righe del dataset di train
titanic_load_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Ottimizzazione del Dataset

Preparazione dei dati rendendoli tutti in forma numerica in quanto il modello della rete neurale di tensorflow esegue calcoli solo su numeri e non sulle stringhe. Inoltre eliminiamo la colonna 'Name' poiché non utile ai fini del problema.

In [0]:
# Codifica di tutti i dati dei dataset titanic_load_train e titanic_load_test
for elem_train in titanic_load_train:
  titanic_load_train[elem_train] = (pd.Categorical(titanic_load_train[elem_train])).codes

for elem_test in titanic_load_test:
  titanic_load_test[elem_test] = (pd.Categorical(titanic_load_test[elem_test])).codes


In [218]:
# Stampa delle prima 5 righe del dataset di train
titanic_load_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,0,2,108,1,28,1,0,523,18,-1,2
1,1,1,0,190,0,51,1,0,596,207,81,0
2,2,1,2,353,0,34,0,0,669,41,-1,2
3,3,1,0,272,0,47,1,0,49,189,55,2
4,4,0,2,15,1,47,0,0,472,43,-1,2


In [219]:
# Stampa delle prima 5 righe del dataset di test
titanic_load_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,2,206,1,44,0,0,152,24,-1,1
1,1,2,403,0,60,1,0,221,5,-1,2
2,2,1,269,1,74,0,0,73,41,-1,1
3,3,2,408,1,34,0,0,147,34,-1,2
4,4,2,178,0,27,1,1,138,46,-1,2


### Ripartizione del Dataset

Si esegue lo Split dei dataset per ricavare i dati di test e di valutazione: 

In [0]:
# Dati di interesse per l'addestramento
features = ["Pclass", "Sex", "SibSp", "Parch"]

In [0]:
# Dividiamo il train set in due parti così da poter addestrare la rete:
# y che contiene solo la colonna 'Survived' che consiste nella soluzione al problema
# X che contiene tutte le altre colonne specificate in 'features'.
y = titanic_load_train.Survived
X = pd.get_dummies(titanic_load_train[features])
X_test = pd.get_dummies(titanic_load_test[features])

# I Dati vengono, a loro volta, divisi in altre due parti: una per
# l'addestramento e una per la valutazione della rete.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2)

# I dati vengono poi memorizzati dentro un Dataset di Tensorflow per sfruttarli
# per l'addestramento. Questa stessa operazione viene poi fatta anche per il
# test set e per l'evaluetion set.
train_dataset = (tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))).shuffle(len(X_train)).batch(1)
val_dataset = (tf.data.Dataset.from_tensor_slices((X_val.values, y_val.values))).shuffle(len(X_val)).batch(1)
test_dataset = (tf.data.Dataset.from_tensor_slices(X_test.values)).batch(1)

## La rete neurale

Il modello utilizzato è formato da 3 layer totalmente connessi e viene utilizzato come algoritmo di ottimizzazione l'adam:

In [0]:
num_inputs = X_train.shape[1]

In [223]:
model = keras.Sequential([
  tf.keras.layers.Dense(32, input_dim=num_inputs, activation='relu'),
  tf.keras.layers.Dense(24, activation='relu'),
  tf.keras.layers.Dense(1)
])

model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 32)                160       
_________________________________________________________________
dense_43 (Dense)             (None, 24)                792       
_________________________________________________________________
dense_44 (Dense)             (None, 1)                 25        
Total params: 977
Trainable params: 977
Non-trainable params: 0
_________________________________________________________________


## Addestramento della rete

Ora è il momento di addestrare il modello tramite i dati di train:

In [224]:
# Addestramento della rete neurale
model.fit(train_dataset, epochs=100)

Epoch 1/100
712/712 [==============================] - 1s 1ms/step - loss: 0.5755 - accuracy: 0.6657
Epoch 2/100
712/712 [==============================] - 1s 1ms/step - loss: 0.4792 - accuracy: 0.7837
Epoch 3/100
712/712 [==============================] - 1s 1ms/step - loss: 0.4616 - accuracy: 0.7851
Epoch 4/100
712/712 [==============================] - 1s 1ms/step - loss: 0.4571 - accuracy: 0.7809
Epoch 5/100
712/712 [==============================] - 1s 1ms/step - loss: 0.4503 - accuracy: 0.7907
Epoch 6/100
712/712 [==============================] - 1s 1ms/step - loss: 0.4429 - accuracy: 0.8076
Epoch 7/100
712/712 [==============================] - 1s 1ms/step - loss: 0.4417 - accuracy: 0.8090
Epoch 8/100
712/712 [==============================] - 1s 1ms/step - loss: 0.4420 - accuracy: 0.7907
Epoch 9/100
712/712 [==============================] - 1s 1ms/step - loss: 0.4418 - accuracy: 0.7978
Epoch 10/100
712/712 [==============================] - 1s 1ms/step - loss: 0.4415 - accura

## Valutazione della rete

Eseguiamo la valutazione del modello:

In [225]:
# Valutazione del modello
val_loss, val_acc = model.evaluate(val_dataset)
print("\nTest accuracy: {:.2f} ({:.2%})".format(val_acc, val_acc))

179/179 [==============================] - 0s 809us/step - loss: 0.4679 - accuracy: 0.7654

Test accuracy: 0.77 (76.54%)


## Previsione

Ora è il momento di predire i valori del dataset di test tramite l'addestramento appena effettuato:

In [0]:
# Predizione dei dati tramite il modello
predictions = model.predict(test_dataset)

Effettuiamo una breve stampa dei valori ottenuti tramite `predict` e degli effettivi valori riferimento. Qui vengono mostrato anche il valore di accuratezza della predizione:

In [227]:
r=0
f=0

# Eseguiamo la stampa dei valori 'predictions' ottenuti tramite
# la predizione con la rete neurale e delle etichette
# reali 'titanic_load_test_label['Survived']'.
print("passegerID \t Prediction \t\t\t Label \t Description\n")
for pid, prediction, label in zip(titanic_load_test['PassengerId'], tf.sigmoid(predictions).numpy(), titanic_load_test_label['Survived']):
  message = "{} \t\t Predicted survival: {:.2%} \t {}".format(pid, prediction[0], label)
  if tf.round(prediction)==label:
    r+=1
  else:
    message += " \t PREDIZIONE ERRATA"
    f+=1
  print(message)

print("\n - Totale predizioni corrette: {}\n - Totale predizioni errate: {}".format(r,f))

passegerID 	 Prediction 			 Label 	 Description

0 		 Predicted survival: 9.94% 	 0
1 		 Predicted survival: 52.64% 	 1
2 		 Predicted survival: 8.34% 	 0
3 		 Predicted survival: 9.94% 	 0
4 		 Predicted survival: 58.09% 	 1
5 		 Predicted survival: 9.94% 	 0
6 		 Predicted survival: 58.75% 	 1
7 		 Predicted survival: 49.26% 	 0
8 		 Predicted survival: 58.75% 	 1
9 		 Predicted survival: 16.31% 	 0
10 		 Predicted survival: 9.94% 	 0
11 		 Predicted survival: 24.70% 	 0
12 		 Predicted survival: 100.00% 	 1
13 		 Predicted survival: 9.94% 	 0
14 		 Predicted survival: 100.00% 	 1
15 		 Predicted survival: 69.76% 	 1
16 		 Predicted survival: 8.34% 	 0
17 		 Predicted survival: 9.94% 	 0
18 		 Predicted survival: 52.64% 	 1
19 		 Predicted survival: 58.75% 	 1
20 		 Predicted survival: 35.69% 	 0
21 		 Predicted survival: 44.06% 	 0
22 		 Predicted survival: 95.79% 	 1
23 		 Predicted survival: 12.69% 	 0
24 		 Predicted survival: 19.45% 	 1 	 PREDIZIONE ERRATA
25 		 Predicted surviv

## Confusion Matrix

Tramite i valori ottenuti è possibile generare una confuion_matrix:



In [0]:
predictions_rounded = tf.round(tf.sigmoid(predictions)).numpy()

In [229]:
print(confusion_matrix(titanic_load_test_label['Survived'], predictions_rounded, normalize=None))

[[257   9]
 [ 10 142]]
